In [ ]:
import numpy as np
import pandas as pd
import torch
from torch import nn
from transformers import XLMRobertaTokenizer, XLMRobertaModel  
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import classification_report
import torch.nn.functional as F
from tqdm import tqdm  

class HateFakeDataset(Dataset):
    def __init__(self, texts, hate_labels, fake_labels, tokenizer, max_len=128):
        self.texts = texts
        self.hate_labels = hate_labels
        self.fake_labels = fake_labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = str(self.texts[idx])
        hate_label = self.hate_labels[idx]
        fake_label = self.fake_labels[idx]

        encoding = self.tokenizer(
            text,
            add_special_tokens=True,
            max_length=self.max_len,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )

        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'hate_label': torch.tensor(hate_label, dtype=torch.long),
            'fake_label': torch.tensor(fake_label, dtype=torch.long)
        }

# Multi-task Model
class MultiTaskHateFakeClassifier(nn.Module):
    def __init__(self, xlm_model, dropout_rate=0.3):
        super().__init__()
        self.xlm = xlm_model
        hidden_size = xlm_model.config.hidden_size

        # Common layers
        self.dropout = nn.Dropout(dropout_rate)
        self.dense1 = nn.Linear(hidden_size, 512)
        self.dense2 = nn.Linear(512, 256)

        # Task-specific layers
        self.hate_classifier = nn.Linear(256, 2)
        self.fake_classifier = nn.Linear(256, 2)

    def forward(self, input_ids, attention_mask):
        outputs = self.xlm(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs.last_hidden_state[:, 0, :]  # CLS token

        x = self.dropout(pooled_output)
        x = F.relu(self.dense1(x))
        x = self.dropout(x)
        x = F.relu(self.dense2(x))
        x = self.dropout(x)

        hate_output = F.softmax(self.hate_classifier(x), dim=1)
        fake_output = F.softmax(self.fake_classifier(x), dim=1)

        return hate_output, fake_output

# Training function 
def train_model(model, train_loader, val_loader, device, num_epochs=30, learning_rate=2e-5, patience=5):
    optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)
    criterion = nn.CrossEntropyLoss()
    best_val_loss = float('inf')
    best_val_accuracy = 0
    early_stop_counter = 0
    best_epoch = 0
    best_model_wts = None

    for epoch in range(num_epochs):
        model.train()
        total_loss = 0

        for batch in tqdm(train_loader, desc=f"Training Epoch {epoch+1}/{num_epochs}", leave=False):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            hate_labels = batch['hate_label'].to(device)
            fake_labels = batch['fake_label'].to(device)

            optimizer.zero_grad()
            hate_output, fake_output = model(input_ids, attention_mask)
            hate_loss = criterion(hate_output, hate_labels)
            fake_loss = criterion(fake_output, fake_labels)
            total_batch_loss = hate_loss + fake_loss

            total_batch_loss.backward()
            optimizer.step()
            total_loss += total_batch_loss.item()

        model.eval()
        val_loss = 0
        all_hate_preds, all_fake_preds = [], []
        all_hate_labels, all_fake_labels = [], []

        with torch.no_grad():
            for batch in tqdm(val_loader, desc="Validating", leave=False):
                input_ids = batch['input_ids'].to(device)
                attention_mask = batch['attention_mask'].to(device)
                hate_labels = batch['hate_label'].to(device)
                fake_labels = batch['fake_label'].to(device)

                hate_output, fake_output = model(input_ids, attention_mask)
                hate_loss = criterion(hate_output, hate_labels)
                fake_loss = criterion(fake_output, fake_labels)
                val_loss += (hate_loss + fake_loss).item()

                all_hate_preds.extend(torch.argmax(hate_output, dim=1).cpu().numpy())
                all_fake_preds.extend(torch.argmax(fake_output, dim=1).cpu().numpy())
                all_hate_labels.extend(hate_labels.cpu().numpy())
                all_fake_labels.extend(fake_labels.cpu().numpy())

        val_hate_accuracy = (torch.argmax(hate_output, dim=1) == hate_labels).float().mean().item()
        val_fake_accuracy = (torch.argmax(fake_output, dim=1) == fake_labels).float().mean().item()
        val_accuracy = (val_hate_accuracy + val_fake_accuracy) / 2

        print(f'Epoch {epoch+1}/{num_epochs}')
        print(f'Training Loss: {total_loss/len(train_loader):.4f}')
        print(f'Validation Loss: {val_loss/len(val_loader):.4f}')
        print(f'Validation Accuracy: {val_accuracy:.4f}')

        print("Classification Report for Hate Speech Task:")
        print(classification_report(all_hate_labels, all_hate_preds))
        print("Classification Report for Fake News Task:")
        print(classification_report(all_fake_labels, all_fake_preds))

        # Check if this is the best epoch
        if val_accuracy > best_val_accuracy:
            best_val_accuracy = val_accuracy
            best_epoch = epoch
            best_model_wts = model.state_dict()  

            # Reset early stop counter since we have a new best model
        #     early_stop_counter = 0
        # else:
        #     early_stop_counter += 1

        # # Early stopping check
        # if early_stop_counter >= patience:
        #     print(f"Early stopping triggered at epoch {epoch+1}.")
        #     break

    model.load_state_dict(best_model_wts)
    torch.save(model.state_dict(), 'best_XLM_model.pt') 

    print(f"Training completed. Best epoch: {best_epoch+1} with validation accuracy: {best_val_accuracy:.4f}")
    print("Best Classification Report for Hate Speech Task:")
    print(classification_report(all_hate_labels, all_hate_preds))
    print("Best Classification Report for Fake News Task:")
    print(classification_report(all_fake_labels, all_fake_preds))

def main():
    df_train = pd.read_csv('A_trans.csv')  
    df_val = pd.read_csv('A_val_trans.csv')  

    for df in [df_train, df_val]:
        df.dropna(subset=['Hate', 'Fake', 'Tweet'], inplace=True)
        df['Hate'] = df['Hate'].astype(int)
        df['Fake'] = df['Fake'].astype(int)
        df = df[(df['Hate'].isin([0, 1])) & (df['Fake'].isin([0, 1]))]

    texts_train = df_train['Tweet'].values
    hate_labels_train = df_train['Hate'].values
    fake_labels_train = df_train['Fake'].values

    texts_val = df_val['Tweet'].values
    hate_labels_val = df_val['Hate'].values
    fake_labels_val = df_val['Fake'].values

    tokenizer = XLMRobertaTokenizer.from_pretrained('xlm-roberta-base')  
    xlm_model = XLMRobertaModel.from_pretrained('xlm-roberta-base')  

    train_dataset = HateFakeDataset(texts_train, hate_labels_train, fake_labels_train, tokenizer)
    val_dataset = HateFakeDataset(texts_val, hate_labels_val, fake_labels_val, tokenizer)

    batch_size = 32
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size)

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = MultiTaskHateFakeClassifier(xlm_model).to(device)

    train_model(model, train_loader, val_loader, device)

if __name__ == '__main__':
    main()


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/615 [00:00<?, ?B/s]

...

Epoch 26/30
Training Loss: 0.9266
Validation Loss: 1.0614
Validation Accuracy: 0.8548
Classification Report for Hate Speech Task:
              precision    recall  f1-score   support

           0       0.68      0.63      0.65       286
           1       0.80      0.83      0.82       513

    accuracy                           0.76       799
   macro avg       0.74      0.73      0.74       799
weighted avg       0.76      0.76      0.76       799

Classification Report for Fake News Task:
              precision    recall  f1-score   support

           0       0.77      0.81      0.79       376
           1       0.82      0.79      0.80       423

    accuracy                           0.80       799
   macro avg       0.80      0.80      0.80       799
weighted avg       0.80      0.80      0.80       799



Epoch 27/30
Training Loss: 0.9140
Validation Loss: 1.0620
Validation Accuracy: 0.8226
Classification Report for Hate Speech Task:
              precision    recall  f1-score   support

           0       0.64      0.68      0.66       286
           1       0.81      0.79      0.80       513

    accuracy                           0.75       799
   macro avg       0.73      0.73      0.73       799
weighted avg       0.75      0.75      0.75       799

Classification Report for Fake News Task:
              precision    recall  f1-score   support

           0       0.78      0.81      0.80       376
           1       0.82      0.80      0.81       423

    accuracy                           0.80       799
   macro avg       0.80      0.80      0.80       799
weighted avg       0.81      0.80      0.80       799



Epoch 28/30
Training Loss: 0.9098
Validation Loss: 1.0594
Validation Accuracy: 0.7903
Classification Report for Hate Speech Task:
              precision    recall  f1-score   support

           0       0.72      0.55      0.62       286
           1       0.78      0.88      0.82       513

    accuracy                           0.76       799
   macro avg       0.75      0.71      0.72       799
weighted avg       0.75      0.76      0.75       799

Classification Report for Fake News Task:
              precision    recall  f1-score   support

           0       0.77      0.82      0.79       376
           1       0.83      0.78      0.81       423

    accuracy                           0.80       799
   macro avg       0.80      0.80      0.80       799
weighted avg       0.80      0.80      0.80       799



Epoch 29/30
Training Loss: 0.9354
Validation Loss: 1.0474
Validation Accuracy: 0.8226
Classification Report for Hate Speech Task:
              precision    recall  f1-score   support

           0       0.69      0.63      0.66       286
           1       0.80      0.84      0.82       513

    accuracy                           0.77       799
   macro avg       0.75      0.73      0.74       799
weighted avg       0.76      0.77      0.76       799

Classification Report for Fake News Task:
              precision    recall  f1-score   support

           0       0.77      0.84      0.80       376
           1       0.84      0.78      0.81       423

    accuracy                           0.80       799
   macro avg       0.81      0.81      0.80       799
weighted avg       0.81      0.80      0.80       799



Epoch 30/30
Training Loss: 0.9399
Validation Loss: 1.0508
Validation Accuracy: 0.8226
Classification Report for Hate Speech Task:
              precision    recall  f1-score   support

           0       0.71      0.65      0.68       286
           1       0.81      0.85      0.83       513

    accuracy                           0.78       799
   macro avg       0.76      0.75      0.75       799
weighted avg       0.77      0.78      0.77       799

Classification Report for Fake News Task:
              precision    recall  f1-score   support

           0       0.75      0.85      0.80       376
           1       0.85      0.75      0.80       423

    accuracy                           0.80       799
   macro avg       0.80      0.80      0.80       799
weighted avg       0.80      0.80      0.80       799

Training completed. Best epoch: 14 with validation accuracy: 0.8548
Best Classification Report for Hate Speech Task:
              precision    recall  f1-score   support

  

In [ ]:
import pandas as pd
import torch
from transformers import XLMRobertaTokenizer, XLMRobertaModel
from torch.utils.data import Dataset, DataLoader
from torch import nn
import torch.nn.functional as F
from tqdm import tqdm  # Import tqdm for progress bars

# Custom Dataset for Test Data
class HateFakeDatasetTest(Dataset):
    def __init__(self, texts, tokenizer, max_len=128):
        self.texts = texts
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = str(self.texts[idx])

        encoding = self.tokenizer(
            text,
            add_special_tokens=True,
            max_length=self.max_len,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )

        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten()
        }

# Multi-task Model Definition (same as before)
class MultiTaskHateFakeClassifier(nn.Module):
    def __init__(self, xlm_model, dropout_rate=0.3):
        super().__init__()
        self.xlm = xlm_model
        hidden_size = xlm_model.config.hidden_size

        # Common layers
        self.dropout = nn.Dropout(dropout_rate)
        self.dense1 = nn.Linear(hidden_size, 512)
        self.dense2 = nn.Linear(512, 256)

        # Task-specific layers
        self.hate_classifier = nn.Linear(256, 2)
        self.fake_classifier = nn.Linear(256, 2)

    def forward(self, input_ids, attention_mask):
        outputs = self.xlm(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs.last_hidden_state[:, 0, :]  # CLS token

        x = self.dropout(pooled_output)
        x = F.relu(self.dense1(x))
        x = self.dropout(x)
        x = F.relu(self.dense2(x))
        x = self.dropout(x)

        hate_output = F.softmax(self.hate_classifier(x), dim=1)
        fake_output = F.softmax(self.fake_classifier(x), dim=1)

        return hate_output, fake_output

# Load the saved model
def load_model(model_path, device):
    tokenizer = XLMRobertaTokenizer.from_pretrained('xlm-roberta-base')  # Use XLM tokenizer
    xlm_model = XLMRobertaModel.from_pretrained('xlm-roberta-base')  # Use XLM model
    model = MultiTaskHateFakeClassifier(xlm_model).to(device)
    model.load_state_dict(torch.load(model_path))
    model.eval()
    return model, tokenizer

# Prediction function
def make_predictions(model, tokenizer, test_loader, device):
    all_predictions = []
    with torch.no_grad():
        # Use tqdm for progress bar with estimated time left
        for batch in tqdm(test_loader, desc="Making Predictions", leave=False, ncols=100, dynamic_ncols=True):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)

            hate_output, fake_output = model(input_ids, attention_mask)

            # Get predicted classes (indices with max probability)
            hate_preds = torch.argmax(hate_output, dim=1).cpu().numpy()
            fake_preds = torch.argmax(fake_output, dim=1).cpu().numpy()

            all_predictions.extend(zip(hate_preds, fake_preds))
    return all_predictions

# Main function for making predictions and saving to CSV
def main():
    # Load the test data
    df_test = pd.read_csv('Test_trans.csv')

    # Initialize tokenizer and device
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    # Load the model
    model, tokenizer = load_model('best_XLM_model.pt', device)

    # Create the test dataset and DataLoader
    test_dataset = HateFakeDatasetTest(df_test['Tweet'].values, tokenizer)
    test_loader = DataLoader(test_dataset, batch_size=32)

    # Make predictions
    predictions = make_predictions(model, tokenizer, test_loader, device)

    # Prepare the result DataFrame
    df_test['Predicted_Hate'] = [pred[0] for pred in predictions]
    df_test['Predicted_Fake'] = [pred[1] for pred in predictions]

    # Save predictions to CSV
    df_test[['Id', 'Predicted_Hate', 'Predicted_Fake']].to_csv('predictions.csv', index=False)
    print("Predictions saved to 'predictions.csv'.")

# Run the main function
if __name__ == '__main__':
    main()


<ipython-input-4-645067230afb>:72: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_path))
                                              

Predictions saved to 'predictions.csv'.


In [ ]:
import numpy as np
import pandas as pd
import torch
from torch import nn
from transformers import XLMRobertaTokenizer, XLMRobertaModel  # Import XLM-R tokenizer and model
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import classification_report
import torch.nn.functional as F
from tqdm import tqdm  # Import tqdm for progress bars

# Custom Dataset
class HateFakeDataset(Dataset):
    def __init__(self, texts, hate_labels, fake_labels, tokenizer, max_len=128):
        self.texts = texts
        self.hate_labels = hate_labels
        self.fake_labels = fake_labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = str(self.texts[idx])
        hate_label = self.hate_labels[idx]
        fake_label = self.fake_labels[idx]

        encoding = self.tokenizer(
            text,
            add_special_tokens=True,
            max_length=self.max_len,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )

        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'hate_label': torch.tensor(hate_label, dtype=torch.long),
            'fake_label': torch.tensor(fake_label, dtype=torch.long)
        }

# Multi-task Model
class MultiTaskHateFakeClassifier(nn.Module):
    def __init__(self, xlm_model, dropout_rate=0.3):
        super().__init__()
        self.xlm = xlm_model
        hidden_size = xlm_model.config.hidden_size

        # Common layers
        self.dropout = nn.Dropout(dropout_rate)
        self.dense1 = nn.Linear(hidden_size, 512)
        self.dense2 = nn.Linear(512, 256)

        # Task-specific layers
        self.hate_classifier = nn.Linear(256, 2)
        self.fake_classifier = nn.Linear(256, 2)

    def forward(self, input_ids, attention_mask):
        outputs = self.xlm(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs.last_hidden_state[:, 0, :]  # CLS token

        x = self.dropout(pooled_output)
        x = F.relu(self.dense1(x))
        x = self.dropout(x)
        x = F.relu(self.dense2(x))
        x = self.dropout(x)

        hate_output = F.softmax(self.hate_classifier(x), dim=1)
        fake_output = F.softmax(self.fake_classifier(x), dim=1)

        return hate_output, fake_output

# Training function with early stopping
def train_model(model, train_loader, val_loader, device, num_epochs=3, learning_rate=2e-5, patience=5):
    optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)
    criterion = nn.CrossEntropyLoss()
    best_val_loss = float('inf')
    best_val_accuracy = 0
    early_stop_counter = 0
    best_epoch = 0
    best_model_wts = None

    for epoch in range(num_epochs):
        model.train()
        total_loss = 0

        for batch in tqdm(train_loader, desc=f"Training Epoch {epoch+1}/{num_epochs}", leave=False):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            hate_labels = batch['hate_label'].to(device)
            fake_labels = batch['fake_label'].to(device)

            optimizer.zero_grad()
            hate_output, fake_output = model(input_ids, attention_mask)
            hate_loss = criterion(hate_output, hate_labels)
            fake_loss = criterion(fake_output, fake_labels)
            total_batch_loss = hate_loss + fake_loss

            total_batch_loss.backward()
            optimizer.step()
            total_loss += total_batch_loss.item()

        model.eval()
        val_loss = 0
        all_hate_preds, all_fake_preds = [], []
        all_hate_labels, all_fake_labels = [], []

        with torch.no_grad():
            for batch in tqdm(val_loader, desc="Validating", leave=False):
                input_ids = batch['input_ids'].to(device)
                attention_mask = batch['attention_mask'].to(device)
                hate_labels = batch['hate_label'].to(device)
                fake_labels = batch['fake_label'].to(device)

                hate_output, fake_output = model(input_ids, attention_mask)
                hate_loss = criterion(hate_output, hate_labels)
                fake_loss = criterion(fake_output, fake_labels)
                val_loss += (hate_loss + fake_loss).item()

                all_hate_preds.extend(torch.argmax(hate_output, dim=1).cpu().numpy())
                all_fake_preds.extend(torch.argmax(fake_output, dim=1).cpu().numpy())
                all_hate_labels.extend(hate_labels.cpu().numpy())
                all_fake_labels.extend(fake_labels.cpu().numpy())

        # Calculate validation accuracy
        val_hate_accuracy = (torch.argmax(hate_output, dim=1) == hate_labels).float().mean().item()
        val_fake_accuracy = (torch.argmax(fake_output, dim=1) == fake_labels).float().mean().item()
        val_accuracy = (val_hate_accuracy + val_fake_accuracy) / 2

        print(f'Epoch {epoch+1}/{num_epochs}')
        print(f'Training Loss: {total_loss/len(train_loader):.4f}')
        print(f'Validation Loss: {val_loss/len(val_loader):.4f}')
        print(f'Validation Accuracy: {val_accuracy:.4f}')

        print("Classification Report for Hate Speech Task:")
        print(classification_report(all_hate_labels, all_hate_preds))
        print("Classification Report for Fake News Task:")
        print(classification_report(all_fake_labels, all_fake_preds))

        # Check if this is the best epoch
        if val_accuracy > best_val_accuracy:
            best_val_accuracy = val_accuracy
            best_epoch = epoch
            best_model_wts = model.state_dict()  # Save the best model weights

            # Reset early stop counter since we have a new best model
        #     early_stop_counter = 0
        # else:
        #     early_stop_counter += 1

        # # Early stopping check
        # if early_stop_counter >= patience:
        #     print(f"Early stopping triggered at epoch {epoch+1}.")
        #     break

    # Load best model weights
    model.load_state_dict(best_model_wts)
    torch.save(model.state_dict(), 'best_XLM_model.pt')  # Save the best model

    print(f"Training completed. Best epoch: {best_epoch+1} with validation accuracy: {best_val_accuracy:.4f}")
    print("Best Classification Report for Hate Speech Task:")
    print(classification_report(all_hate_labels, all_hate_preds))
    print("Best Classification Report for Fake News Task:")
    print(classification_report(all_fake_labels, all_fake_preds))

# Usage example
def main():
    # Load and preprocess your data
    df_train = pd.read_csv('A_trans.csv')  # Load whole training data
    df_val = pd.read_csv('/content/Test_trans_predictions.csv')  # Load validation data

    # Clean and preprocess the labels
    for df in [df_train, df_val]:
        df.dropna(subset=['Hate', 'Fake', 'Tweet'], inplace=True)
        df['Hate'] = df['Hate'].astype(int)
        df['Fake'] = df['Fake'].astype(int)
        df = df[(df['Hate'].isin([0, 1])) & (df['Fake'].isin([0, 1]))]

    # Extract texts and labels for training and validation
    texts_train = df_train['Tweet'].values
    hate_labels_train = df_train['Hate'].values
    fake_labels_train = df_train['Fake'].values

    texts_val = df_val['Tweet'].values
    hate_labels_val = df_val['Hate'].values
    fake_labels_val = df_val['Fake'].values

    # Initialize tokenizer and model for XLM-R
    tokenizer = XLMRobertaTokenizer.from_pretrained('xlm-roberta-base')  # Use XLM-R tokenizer
    xlm_model = XLMRobertaModel.from_pretrained('xlm-roberta-base')  # Use XLM-R model

    # Create datasets
    train_dataset = HateFakeDataset(texts_train, hate_labels_train, fake_labels_train, tokenizer)
    val_dataset = HateFakeDataset(texts_val, hate_labels_val, fake_labels_val, tokenizer)

    # Create DataLoaders
    batch_size = 32
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size)

    # Initialize model
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = MultiTaskHateFakeClassifier(xlm_model).to(device)

    # Train model
    train_model(model, train_loader, val_loader, device)

# Run the main function
if __name__ == '__main__':
    main()


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Epoch 1/3
Training Loss: 1.3281
Validation Loss: 1.1538
Validation Accuracy: 0.8125
Classification Report for Hate Speech Task:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00       249
           1       0.69      1.00      0.82       551

    accuracy                           0.69       800
   macro avg       0.34      0.50      0.41       800
weighted avg       0.47      0.69      0.56       800

Classification Report for Fake News Task:
              precision    recall  f1-score   support

           0       0.92      0.98      0.95       422
           1       0.97      0.91      0.94       378

    accuracy                           0.94       800
   macro avg       0.95      0.94      0.94       800
weighted avg       0.95      0.94      0.94       800



Epoch 2/3
Training Loss: 1.1976
Validation Loss: 0.9798
Validation Accuracy: 0.8438
Classification Report for Hate Speech Task:
              precision    recall  f1-score   support

           0       0.45      0.59      0.51       249
           1       0.78      0.67      0.72       551

    accuracy                           0.65       800
   macro avg       0.62      0.63      0.62       800
weighted avg       0.68      0.65      0.66       800

Classification Report for Fake News Task:
              precision    recall  f1-score   support

           0       0.96      0.98      0.97       422
           1       0.98      0.95      0.96       378

    accuracy                           0.97       800
   macro avg       0.97      0.97      0.97       800
weighted avg       0.97      0.97      0.97       800



Epoch 3/3
Training Loss: 1.1474
Validation Loss: 0.9266
Validation Accuracy: 0.9062
Classification Report for Hate Speech Task:
              precision    recall  f1-score   support

           0       0.60      0.35      0.44       249
           1       0.75      0.89      0.82       551

    accuracy                           0.72       800
   macro avg       0.67      0.62      0.63       800
weighted avg       0.70      0.72      0.70       800

Classification Report for Fake News Task:
              precision    recall  f1-score   support

           0       0.99      0.95      0.97       422
           1       0.95      0.98      0.96       378

    accuracy                           0.97       800
   macro avg       0.97      0.97      0.97       800
weighted avg       0.97      0.97      0.97       800

Training completed. Best epoch: 3 with validation accuracy: 0.9062
Best Classification Report for Hate Speech Task:
              precision    recall  f1-score   support

     